standardization

In [2]:
from unidecode import unidecode
def to_english(s):
    s = list(str(s))
    for i in range (len(s)):
        s[i] = unidecode(s[i])
    s1 = ""
    for i in s:
        s1+=i
    s1 = s1.lower()
    s1 = s1.replace(".","")
    s1 = s1.replace("ha noi","hn")
    s1 = s1.replace("city","")
    s1 = s1.replace("thanh pho","")
    s1 = s1.replace("ho chi minh","hcm")
    s1 = s1.replace("tp","")
    s1 = s1.replace("-","")
    return " ".join(s1.split())

making trainning file

In [4]:
import pandas as pd
import fasttext
file = pd.read_csv("school_list_fasttext.csv",)
with open("school_train.txt","w",encoding = 'utf-8') as f:
    for i in range(len(file)):
        for column in file:
            if column == "__label__":
                continue
            if not pd.isna(file.iloc[i][column]):
                f.write(file.iloc[i]["__label__"]+ " " + to_english(file.iloc[i][column].lower()) +"\n")
    f.close()
            

making model

In [ ]:
import fasttext
model = fasttext.train_supervised(input = "school_train.txt",lr = 1.0,epoch = 15)

In [ ]:
import pandas as pd
data = pd.read_csv('school_major.csv')
lst1 = []
lst2 = []
for ind in data.index:
    n = to_english(" ".join(str(data.loc[ind]["school_name"]).split()))
    lst1.append(n)
    if "trung hoc" in n or "thcs" in n or "thpt" in n or "ptth" in n or "trung cap" in n or "gdtx" in n or "giao duc thuong xuyen" in n:
        lst2.append("Trung học")
    else:
        lst2.append(model.predict(n))
data["normalized"] = lst1
data["predicted"] = lst2
print(data)

Self-generating training file

In [ ]:
c = pd.read_csv("convert_label_school.csv")
convert = dict()
for ind in c.index:
    convert[c.loc[ind]["__label__"]] = c.loc[ind]["school"]
convert["Trung hoc"] = "trung hoc"
lst1 = []
lst2 = []
for ind in data.index:
    if data.loc[ind]["predicted"] == "Trung học":
        lst1.append("trung hoc")
        lst2.append("100")
        continue
    lst1.append(convert[data.loc[ind]["predicted"][0][0]])
    lst2.append(data.loc[ind]["predicted"][1][0])
data["school_predict"] = lst1
data["confidence"] = lst2
data.drop("predicted", axis = 1, inplace=True)
print(data)

In [ ]:
data.to_csv("testing.csv")

test_data = pd.read_csv("finalized.csv")
test_data.drop("Unnamed: 0",axis = 1,inplace = True)
print(test_data)

Model using example.csv

In [37]:
test_data = pd.read_csv("example.csv")
test_data.drop("Unnamed: 0",axis = 1,inplace = True)

In [38]:
lst1 = []
lst2 = []
for ind in test_data.index:
    test_acc = test_data.loc[ind]["predicted_school"].split(",")
    lst1.append(float(test_acc[0].replace("(","")))
    lst2.append(to_english(test_acc[1].replace(")","").replace("'","")))
test_data = test_data.assign(accuracy_school = lst1,)
test_data = test_data.assign(school_predict = lst2,)
print(test_data)

                                school_name            specialization  \
0                                    DHBKHN                      CNTT   
1                    vien dai hoc mo ha noi  khoa công nghệ thông tin   
2                    Viện đại học Mở Hà Nội  Khoa công nghệ thông tin   
3                                       Fpt                   Android   
4                     Dh công nghiệp hà nội       công nghệ thông tin   
...                                     ...                       ...   
9994                       Đại học Thủy Lợi              Chuyên ngành   
9995                       dai hoc thuy loi              Chuyên ngành   
9996                        đại học luật                 hình sự   
9997  trường trung cấp nghề cơ khí 1 hà nội      ngành  nguội chế tạo   
9998              đại học công nghệ sài gòn       quản trị kinh doanh   

                   normalized_school  \
0               đại học bách khoa hn   
1             vien đại học mo ha noi   
2  

In [39]:
import random
test_data = test_data[test_data.accuracy_school>0.7].reset_index(drop = True)
train = dict()
train["school_name"] = []
train["school_predict"] = []
test = dict()
test["school_name"] = []
test["school_predict"] = []
for ind in test_data.index:
    num = random.randint(1,4)
    if num == 1:
        test["school_name"].append(to_english(test_data.loc[ind]["school_name"]))
        test["school_predict"].append("__label__" + to_english(test_data.loc[ind]["school_predict"]).replace(" ","-"))
    else:
        train["school_name"].append(to_english(test_data.loc[ind]["school_name"]))
        train["school_predict"].append("__label__" + to_english(test_data.loc[ind]["school_predict"]).replace(" ","-"))
testing_set = pd.DataFrame(test)
training_set = pd.DataFrame(train)
with open("training_set.txt",'w',encoding = 'utf-8') as f:
    for i in range(len(train["school_predict"])):
        f.write(train["school_predict"][i] + " " + train["school_name"][i] + "\n")
    f.close()
with open("testing_set.txt",'w',encoding = 'utf-8') as f:
    for i in range(len(test["school_predict"])):
        f.write(test["school_predict"][i] + " " + test["school_name"][i] + "\n")
    f.close()
testing_set.to_csv("testing_set.csv")
training_set.to_csv("training_set.csv")

In [40]:
model_x = fasttext.train_supervised(input="training_set.txt",lr = 1.0, epoch = 25)

In [43]:
model_x.test("testing_set.txt")

(1632, 0.9675245098039216, 0.9675245098039216)

In [42]:
model_x.save_model("model_ex.bin")

In [ ]:
model using finalized.csv

In [44]:
test_data = pd.read_csv("finalized.csv")
test_data.drop("Unnamed: 0",axis = 1,inplace = True)
test_data = test_data[(test_data['school_p_score'] > 90) | (test_data['school_r_score']>95)]
train = dict()
train["school_name"] = []
train["school_predict"] = []
test = dict()
test["school_name"] = []
test["school_predict"] = []
for ind in test_data.index:
    num = random.randint(1,4)
    if num == 1:
        test["school_name"].append(to_english(test_data.loc[ind]["school"]))
        test["school_predict"].append("__label__" + to_english(test_data.loc[ind]["school_standard"]).replace(" ","-"))
    else:
        train["school_name"].append(to_english(test_data.loc[ind]["school"]))
        train["school_predict"].append("__label__" + to_english(test_data.loc[ind]["school_standard"]).replace(" ","-"))
with open("training_set_f.txt",'w',encoding = 'utf-8') as f:
    for i in range(len(train["school_predict"])):
        f.write(train["school_predict"][i] + " " + train["school_name"][i] + "\n")
    f.close()
with open("testing_set_f.txt",'w',encoding = 'utf-8') as f:
    for i in range(len(test["school_predict"])):
        f.write(test["school_predict"][i] + " " + test["school_name"][i] + "\n")
    f.close()
print(test_data)

                         school                 school_norm  \
0                        DHBKHN    dai hoc bach khoa ha noi   
1        vien dai hoc mo ha noi      vien dai hoc mo ha noi   
2        Viện đại học Mở Hà Nội      vien dai hoc mo ha noi   
3                           Fpt                         fpt   
4         Dh công nghiệp hà nội  dai hoc cong nghiep ha noi   
...                         ...                         ...   
9992    cao dang Co Dien Ha Noi     cao dang co dien ha noi   
9994           Đại học Thủy Lợi            dai hoc thuy loi   
9995           dai hoc thuy loi            dai hoc thuy loi   
9996            đại học luật                dai hoc luat   
9998  đại học công nghệ sài gòn   dai hoc cong nghe sai gon   

                                school_standard            specialization  \
0                      Đại học Bách khoa Hà Nội                      CNTT   
1                      Trường Đại học Mở Hà Nội  khoa công nghệ thông tin   
2           

In [45]:
model_f = fasttext.train_supervised(input = 'training_set_f.txt',lr = 1.0, epoch = 25)

In [47]:
model_f.test('testing_set_f.txt')

(1943, 0.9660319094184251, 0.9660319094184251)

In [49]:
model_f.save_model("model_f.bin")

Experimenting

In [ ]:
convert_test_fasttext = dict()
for school in test_data["school_predict"].unique():
    n = model.predict(school)
    convert_test_fasttext[school] = [n[0][0],n[1][0]]
additional_data = pd.DataFrame(convert_test_fasttext.values(),index = convert_test_fasttext.keys())
additional_data = additional_data.rename(columns={0:'label',1:'score'})
additional_data.to_csv("additional_data.csv")
print(additional_data)

In [ ]:
import pandas as pd
new_data = pd.read_csv("new_data.csv")
new_data.dropna(inplace=True)
with open("school_train.txt","a",encoding = 'utf-8') as f:
    f.write("\n")
    for ind in new_data.index:
        f.write(new_data.loc[ind]["__label__"] + " " + new_data.loc[ind]["school"] + "\n")
    f.close()
print(new_data)

In [ ]:
with open("s2.txt",'w',encoding = 'utf-8') as f:
    school_track = []
    for ind in test_data.index:
        if to_english(test_data.loc[ind]["school_name"]) in school_track:
            continue        found = False
        track = 0
        school_track.append(to_english(test_data.loc[ind]["school_name"]))
        for ind_s in new_data.index:
            if test_data.loc[ind]["school_predict"] == new_data.loc[ind_s]["school"]:
                track = ind_s
                found = True
                break
        if found and test_data.loc[ind]["accuracy_school"] >= 0.90:
            f.write(new_data.loc[ind_s]["__label__"] + " " + to_english(test_data.loc[ind]["school_name"]) + "\n")
    f.close()
            

In [ ]:
check = set()
with open("school_train.txt",'r',encoding = 'utf-8') as f:
    for line in f.readlines():
        check.add(line)
    f.close()
with open("s2.txt",'w',encoding = 'utf-8') as f:
    for line in check:
        f.write(line)
    f.close()
print(len(check))
print(check)